In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))  # sobe um nível a partir da pasta models/

In [2]:
import ast
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

from copy import deepcopy
from sklearn.model_selection import KFold

from val.Wrapper import ForwardFeatureSelector

In [3]:
class MLP_100_100_100_regressor(nn.Module):
    def __init__(self, input_len):
        super().__init__()

        self.layer1 = nn.Linear(input_len, 100)  
        self.relu = nn.ReLU()
        self.hidden_layer_relu = nn.Sequential(
            nn.Linear(100,100),
            nn.ReLU(),
            nn.Linear(100,100),
            nn.ReLU(),
            nn.Linear(100,100),
            nn.ReLU()
        )                          
        self.layer2 = nn.Linear(100, 30) 

    def forward(self, x):

        x = self.relu(self.layer1(x))
        x = self.hidden_layer_relu(x)
        logits = self.layer2(x)
        
        return logits

In [4]:
# abrindo os dados de treinamento
df = pd.read_csv("./../ansatz_result/data.csv")
X = df.drop(columns=["target"]).to_numpy()
y = pd.DataFrame(df['target'].apply(ast.literal_eval).tolist()).to_numpy()
y_regression = y.astype(np.float32)  # Garanta que é float32

# criando o kfold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

In [5]:
print(y)

[[0.7        0.5        0.72857143 ... 0.71428571 0.72857143 0.71428571]
 [0.72857143 0.5        0.7        ... 0.68571429 0.7        0.71428571]
 [0.75714286 0.5        0.75714286 ... 0.75714286 0.77142857 0.77142857]
 ...
 [0.52285714 0.5        0.50857143 ... 0.50571429 0.50571429 0.50857143]
 [0.5        0.5        0.49285714 ... 0.50428571 0.49857143 0.5       ]
 [0.51714286 0.5        0.52428571 ... 0.49       0.52142857 0.50428571]]


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Using cpu device


In [7]:
# implementação de um critério de parada para o modelo parar no ponto "ótimo"
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False
        
    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [8]:
# 3. Função para treinar um fold
def train_fold(model, train_loader, val_loader, criterion, optimizer, n_epochs=1000000):
    model.to(device)
    early_stopping = EarlyStopping(patience=10, min_delta=0.0)
    best_model = None
    best_val_loss = 1000000
    
    for epoch in range(n_epochs):
        # Treinamento
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_batch = y_batch.squeeze()  # Converte (batch_size, 1) para (batch_size,)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validação
        model.eval()
        with torch.no_grad():
            val_loss = 0.0

            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                outputs = model(X_val)
                val_loss += criterion(outputs, y_val).item()
        
        
        val_loss /= len(val_loader)
        
        print(f'Epoch {epoch+1}/{n_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} ')
        
        # Early Stopping
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
        # Salvar melhor modelo
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = deepcopy(model.state_dict())
    
    return best_model, best_val_loss

In [9]:
results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n=== Fold {fold + 1}/{3} ===")
    
    # Dividir e preparar dados
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_regression[train_idx], y_regression[val_idx]
    
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)
    
    # Inicializar modelo e otimizador
    model = MLP_100_100_100_regressor(22)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    
    # Treinar fold
    best_model, best_mse = train_fold(model, train_loader, val_loader, criterion, optimizer)
    results.append(best_mse)
    
    # Salvar modelo se necessário
    torch.save(best_model, f'./models_salvos/best_model_MLP-100-100-100-regressor_fold{fold+1}.pt')

# 7. Resultados finais
print("\n=== Resultados ===")
print(f" MSE médio: {np.mean(results):.4f} ± {np.std(results):.4f}")
print(f"MSE por fold: {results}")


=== Fold 1/3 ===
Epoch 1/1000000 - Train Loss: 1.2163 - Val Loss: 0.0262 
Epoch 2/1000000 - Train Loss: 0.0956 - Val Loss: 0.0068 
Epoch 3/1000000 - Train Loss: 0.0530 - Val Loss: 0.0051 
Epoch 4/1000000 - Train Loss: 0.0401 - Val Loss: 0.0044 
Epoch 5/1000000 - Train Loss: 0.0330 - Val Loss: 0.0034 
Epoch 6/1000000 - Train Loss: 0.0255 - Val Loss: 0.0025 
Epoch 7/1000000 - Train Loss: 0.0168 - Val Loss: 0.0020 
Epoch 8/1000000 - Train Loss: 0.0144 - Val Loss: 0.0015 
Epoch 9/1000000 - Train Loss: 0.0111 - Val Loss: 0.0014 
Epoch 10/1000000 - Train Loss: 0.0113 - Val Loss: 0.0014 
Epoch 11/1000000 - Train Loss: 0.0113 - Val Loss: 0.0013 
Epoch 12/1000000 - Train Loss: 0.0106 - Val Loss: 0.0013 
Epoch 13/1000000 - Train Loss: 0.0102 - Val Loss: 0.0012 
Epoch 14/1000000 - Train Loss: 0.0100 - Val Loss: 0.0013 
Epoch 15/1000000 - Train Loss: 0.0098 - Val Loss: 0.0012 
Epoch 16/1000000 - Train Loss: 0.0095 - Val Loss: 0.0012 
Epoch 17/1000000 - Train Loss: 0.0091 - Val Loss: 0.0012 
Epoch

In [10]:
from sklearn.metrics import mean_squared_error

selector = ForwardFeatureSelector(
    model=lambda input_dim: MLP_100_100_100_regressor(input_dim),
    model_type='pytorch',
    scoring=mean_squared_error,
    cv=3,
    verbose=1,
    pytorch_optimizer= optim.Adam,
    classification=0,
    maior_score=0
)

In [11]:
X_new = selector.fit_transform(X, y)

Testing feature set: [0]
Testing feature set: [1]
Testing feature set: [2]
Testing feature set: [3]
Testing feature set: [4]
Testing feature set: [5]
Testing feature set: [6]
Testing feature set: [7]
Testing feature set: [8]
Testing feature set: [9]
Testing feature set: [10]
Testing feature set: [11]
Testing feature set: [12]
Testing feature set: [13]
Testing feature set: [14]
Testing feature set: [15]
Testing feature set: [16]
Testing feature set: [17]
Testing feature set: [18]
Testing feature set: [19]
Testing feature set: [20]
Testing feature set: [21]
0
0.2801286014402799
Selected feature: 0 | Score: 0.2801
Testing feature set: [0, 1]
Testing feature set: [0, 2]
Testing feature set: [0, 3]
Testing feature set: [0, 4]
Testing feature set: [0, 5]
Testing feature set: [0, 6]
Testing feature set: [0, 7]
Testing feature set: [0, 8]
Testing feature set: [0, 9]
Testing feature set: [0, 10]
Testing feature set: [0, 11]
Testing feature set: [0, 12]
Testing feature set: [0, 13]
Testing featu